In [1]:
import requests
#r = requests.post('http://10.30.99.211:8261/gpt_small', data = "What is better for deep learning Python or Matlab?")
#print (r.status_code)

In [2]:
! pwd

/notebook/touche


In [ ]:
#numpy
import numpy as np

#torch
import torch, torch.nn as nn
import torch.nn.functional as F

#bert service
from bert_serving.client import BertClient
from sklearn.metrics.pairwise import cosine_similarity
bc = BertClient()

# transformers
import pytorch_transformers
from pytorch_transformers import *



In [ ]:
import pickle
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
from xml.dom import minidom
import re

def read_xml(filename):
    # convert file filename to list of tuples (number_of_topic, title_of_topic) 
    # input: filename string
    # output: list of corresponding tuples
    answer_list = []
    xmldoc = minidom.parse(filename)
    itemlist = xmldoc.getElementsByTagName('topics')
    print(len(itemlist))
    print(itemlist)
    topic_list = itemlist[0].getElementsByTagName('topic')
    print (len(topic_list))
    for topic in topic_list:
        tuple_for_add = tuple((topic.getElementsByTagName('number')[0].firstChild.nodeValue, topic.getElementsByTagName('title')[0].firstChild.nodeValue))
        answer_list.append(tuple_for_add)
    return answer_list

def make_a_search_request(query, size=10):
    # return json
    # json will be processed further
    params = {
            "apikey": "0833a307-97d3-462a-99d9-27db400c70da",
            "query": query,
            "index": ["cw12"],
            "size": size,
            "pretty": True
        }
    response = requests.get(url = "http://www.chatnoir.eu/api/v1/_search", params = params)
    return response

def clean_punct(s):
    s = re.sub(r'[^\w\s]','',s)
    return s

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext.lower()


In [ ]:
def make_scores_1(query, answer_titles):
    query_emb = bc.encode([query])
    scores = [cosine_similarity(query_emb.reshape(1, -1), bc.encode([cleanhtml(answer_title)]).reshape(1, -1))[0][0] for answer_title in answer_titles]
    return scores

In [ ]:
def make_scores_transformers(model, query, answers_bodies):
    scores = []
    for ind, elem in enumerate(answers_bodies):
        sentences = ["[CLS] " + query + " [SEP] " + answers_bodies[ind] + " [SEP]"]
        #print(sentences[0])

        # Tokenize with BERT tokenizer
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
        tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
        #print ("Tokenize the first sentence:")
        #print (tokenized_texts[0])
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_texts[0])
        # Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
        segments_ids = (len(tokenizer.tokenize(query)) + 2)*[0] + (len(tokenizer.tokenize(answers_bodies[ind])) + 1)*[1]
        cls_idx = 0
        sep_idx = len(tokenizer.tokenize(query))
        end_idx = len(tokenizer.tokenize(query)) + len(tokenizer.tokenize(answers_bodies[ind])) + 3
        #print (len(tokenizer.tokenize(query)), len(tokenizer.tokenize(answers_bodies[ind])))
        #print (len(indexed_tokens), len(segments_ids))
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])
        answ = model(tokens_tensor, segments_tensors)

        attensions = answ[2][3][0][0].detach().numpy() + answ[2][9][0][0].detach().numpy()
        #probs[cls_idx:sep_idx, sep_idx:] = 0.0
        score = np.sum(attensions[cls_idx:sep_idx, sep_idx:]) + np.sum(attensions[sep_idx:end_idx, cls_idx:sep_idx+2])
        #print (score)
        scores.append(score)
    return scores
    

In [ ]:
def run_baseline(output_dir = '/notebook/touche/output/', input_dir = '/notebook/touche/', input_file = 'topics-task-2-only-titles.xml'):
    
    #transformer part
    config_class, model_class, tokenizer_class = BertConfig, BertModel, BertTokenizer
    config = config_class.from_pretrained('bert-base-uncased')
    config.output_attentions=True
    model = model_class.from_pretrained('bert-base-uncased', config=config)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    
    # load responses for all queries from file 
    my_response_list = load_obj("list_of_un_answ")
    
    list_of_tuples = read_xml(input_dir + input_file)
    common_list = []
    
        
    print ("attention", flush = True)    
    with open(output_dir + 'transformer.txt', 'w') as trnsfm:
        for ind_q, elem in enumerate(list_of_tuples[:5]):
            qid = elem[0]
            Q0 = 'Q0'
            query = elem[1]
            tag = 'MethodAttentionFilterResponse'
            responses = list(zip(*my_response_list[str(ind_q + 1)]))

            scores0 = responses[0]
            print ("0")
            docs = responses[1]
            print ("1")
            titles = responses[2]
            #print ("2")
            answers_bodies = responses[3]
            #print ("3")
            # print (scores0, scores3, scores)
            scores = make_scores_transformers(model, query, answers_bodies)
            qids = qid*len(scores)
            Q0s = [Q0 for elem in scores]
            queries = query*len(scores)
            tags = [tag for elem in scores]
            
            help_list = list(zip(qids, Q0s, queries, scores, titles, answers_bodies))
            help_list = sorted(help_list, key = lambda x: x[3], reverse = True)
            trnsfm.write('\n'.join('%s %s %s %s %s %s' % x for x in help_list))
            trnsfm.write('\n')
            
            part_of_commom_list = list(zip(qids, Q0s, docs, scores, tags))
            part_of_commom_list = sorted(part_of_commom_list, key = lambda x: x[3], reverse = True) 
            

            qids, Q0s, docs, scores, tags = zip(*part_of_commom_list)

            ranks = range(1, len(scores) + 1)
            part_of_commom_list = list(zip(qids, Q0s, docs, ranks, scores, tags))
            common_list = common_list + part_of_commom_list
        
    print ("baseline", flush = True)
    with open(output_dir + 'baseline.txt', 'w') as bsln:
        for ind_q, elem in enumerate(list_of_tuples[:3]):
            qid = elem[0]
            Q0 = 'Q0'
            query = elem[1]
            tag = 'MyBaselineFilterResponse'
            responses = list(zip(*my_response_list[str(ind_q + 1)]))

            scores0 = responses[0]
            #print ("0")
            docs = responses[1]
            #print ("1")
            titles = responses[2]
            #print ("2")
            answers_bodies = responses[3]
            #print ("3")
            # print (scores0, scores3, scores)
            qids = qid*len(scores)
            Q0s = [Q0 for elem in scores]
            tags = [tag for elem in scores]
            part_of_commom_list = list(zip(qids, Q0s, docs, scores0, tags))
            part_of_commom_list = sorted(part_of_commom_list, key = lambda x: x[3], reverse = True) 

            qids, Q0s, docs, scores, tags = zip(*part_of_commom_list)
            
            help_list = list(zip(qids, Q0s, queries, scores, titles, answers_bodies))
            help_list = sorted(help_list, key = lambda x: x[3], reverse = True)
            bsln.write('\n'.join('%s %s %s %s %s %s' % x for x in help_list))
            bsln.write('\n')

            ranks = range(1, len(scores) + 1)
            part_of_commom_list = list(zip(qids, Q0s, docs, ranks, scores, tags))
            common_list = common_list + part_of_commom_list
            part_of_commom_list = list(zip(qids, Q0s, docs, ranks, scores, tags))
            common_list = common_list + part_of_commom_list
        
    print ("bert embeddings", flush = True)
    with open(output_dir + 'bert_e.txt', 'w') as bert_e:
        for ind_q, elem in enumerate(list_of_tuples[:5]):
            qid = elem[0]
            Q0 = 'Q0'
            query = elem[1]
            tag = 'myBertSimilarity'
            responses = list(zip(*my_response_list[str(ind_q + 1)]))

            scores0 = responses[0]
            #print ("0")
            docs = responses[1]
            #print ("1")
            titles = responses[2]
            #print ("2")
            answers_bodies = responses[3]
            #print ("3")
            # print (scores0, scores3, scores)
            scores = make_scores_1(query, titles)
            qids = qid*len(scores)
            Q0s = [Q0 for elem in scores]
            tags = [tag for elem in scores]
            part_of_commom_list = list(zip(qids, Q0s, docs, scores0, tags))
            part_of_commom_list = sorted(part_of_commom_list, key = lambda x: x[3], reverse = True) 

            qids, Q0s, docs, scores, tags = zip(*part_of_commom_list)

            help_list = list(zip(qids, Q0s, queries, scores, titles, answers_bodies))
            help_list = sorted(help_list, key = lambda x: x[3], reverse = True)
            bert_e.write('\n'.join('%s %s %s %s %s %s' % x for x in help_list))
            bert_e.write('\n')

            ranks = range(1, len(scores) + 1)
            part_of_commom_list = list(zip(qids, Q0s, docs, ranks, scores, tags))
            common_list = common_list + part_of_commom_list
            part_of_commom_list = list(zip(qids, Q0s, docs, ranks, scores, tags))
            common_list = common_list + part_of_commom_list

    with open(output_dir + 'run_example5_22_05.txt', 'w') as fp:
        fp.write('\n'.join('%s %s %s %s %s %s' % x for x in common_list))

In [ ]:
config_class, model_class, tokenizer_class = BertConfig, BertModel, BertTokenizer
config = config_class.from_pretrained('bert-base-uncased')
config.output_attentions=True
model = model_class.from_pretrained('bert-base-uncased', config=config)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
import bokeh.plotting as pl
import bokeh.models as bm
from bokeh.io import output_notebook, show
output_notebook()

def draw_attention(inp_tokens, trans_tokens, probs):
    """ An intentionally ambiguous function to visualize attention weights """
    probs = probs[:len(trans_tokens), :len(inp_tokens)]
    
    fig = pl.figure(x_range=(0, len(inp_tokens)), y_range=(0, len(trans_tokens)),
                    x_axis_type=None, y_axis_type=None, tools=[])
    fig.image([probs[::-1]], 0, 0, len(inp_tokens), len(trans_tokens))

    fig.add_layout(bm.LinearAxis(axis_label='source tokens'), 'above')
    fig.xaxis.ticker = np.arange(len(inp_tokens)) + 0.5
    fig.xaxis.major_label_overrides = dict(zip(np.arange(len(inp_tokens)) + 0.5, inp_tokens))
    fig.xaxis.major_label_orientation = 45

    fig.add_layout(bm.LinearAxis(axis_label='translation tokens'), 'left')
    fig.yaxis.ticker = np.arange(len(trans_tokens)) + 0.5
    fig.yaxis.major_label_overrides = dict(zip(np.arange(len(trans_tokens)) + 0.5, trans_tokens[::-1]))

    show(fig)

In [ ]:
np.arange(len(inp_tokens)) + 0.5

Идея - из аттеншн слоев выбрать те, которые в левом вернем и правом нижнем углу (0 предложение на себя и  предложение на себя) не дают полностью черных квадратов. 3 слой - атеншн на аналогичные слова

обнулить sep-tokena

Просуммировать левый верхний и правый нижний квадрат - для каждого поискового ответа

In [ ]:
probs = answ[2][ii][0][0].detach().numpy()

probs = probs[:len(trans_tokens[0]), :len(inp_tokens[0])]

In [ ]:
probs

In [ ]:
cls_idx = 0
sep_idx = len(tokenizer.tokenize(query))
end_idx = len(tokenizer.tokenize(query)) + len(tokenizer.tokenize(answers_bodies[10])) + 3

In [ ]:
probs = answ[2][ii][0][0].detach().numpy()
probs = probs[cls_idx:sep_idx, sep_idx:end_idx]
probs.shape

In [ ]:
len(trans_tokens[0][cls_idx:sep_idx])

In [ ]:
query = 'Which is better, a laptop or a desktop?'
answer1 = 'Laptop is preferable than desktop because it is more portable.'
answer2 = 'If you need portability, the laptop is the best option than desktop.'
answer3 = 'The crew was complete: it included a boots, a maker of bonnets and hoods'
answer4 = 'Just the pace for snark the Bellman cried'

Expected results: answer1 and answer2 are better than answer3

In [ ]:
from bokeh.layouts import row, column
def show_importance_of_transformers_layer(query, answer):

    # return scores - list of len = number_of_transformers_heads
    # every element is sum of weight in left top and rinht bottom square - attention weights which correspond to tokens 
    # not in word's sentence. 
    # To estimate closeness between sentences A and B for every word in A we count all weight corresponding to word in B

    sentences = ["[CLS] " + query + " [SEP] " + answer + " [SEP]"]
    print(sentences[0])

    # Tokenize with BERT tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    print ("Tokenize the first sentence:")
    print (tokenized_texts[0])
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_texts[0])
    # Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
    segments_ids = (len(tokenizer.tokenize(query)) + 2)*[0] + (len(tokenizer.tokenize(answer)) + 1)*[1]
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    answ = model(tokens_tensor, segments_tensors)
    trans_tokens = tokenized_texts
    inp_tokens = tokenized_texts
    cls_idx = 0
    sep_idx = len(tokenizer.tokenize(query))
    end_idx = len(tokenizer.tokenize(query)) + len(tokenizer.tokenize(answer)) + 3

    print (cls_idx, sep_idx, end_idx)
    scores = []
    list_of_figures = []
    for ii in range(12):
        probs = answ[2][ii][0][0].detach().numpy()

        probs = probs[:len(trans_tokens[0]), :len(inp_tokens[0])]
        score = np.sum(probs[cls_idx+1:sep_idx-1, sep_idx+1:end_idx -1]) + np.sum(probs[sep_idx+1:end_idx-1, cls_idx+1:sep_idx-2])
        fig = pl.figure(x_range=(0, len(inp_tokens[0])), y_range=(0, len(trans_tokens[0])),
                        x_axis_type=None, y_axis_type=None, tools=[], plot_width=250, plot_height=250, title = 'head number' + str(ii) + ' score ' + str(score))
        fig.image([probs], 0, 0, len(inp_tokens[0]), len(trans_tokens[0]))
        scores.append(score)
        list_of_figures.append(fig)
        #fig.add_layout(bm.LinearAxis(axis_label='source tokens'), 'above')
        #fig.xaxis.ticker = np.arange(len(inp_tokens[0])) + 0.5
        #fig.xaxis.major_label_overrides = dict(zip(np.arange(len(inp_tokens[0])) + 0.5, inp_tokens[0]))
        #fig.xaxis.major_label_orientation = 45

        #fig.add_layout(bm.LinearAxis(axis_label='translation tokens'), 'left')
        #fig.yaxis.ticker = np.arange(len(trans_tokens[0])) + 0.5
        #fig.yaxis.major_label_overrides = dict(zip(np.arange(len(trans_tokens[0])) + 0.5, trans_tokens[0]))

    show(row(list_of_figures[:4]))
    show(row(list_of_figures[4:8]))
    show(row(list_of_figures[8:12]))
    return scores

**Что берем?**

**3 слой** - я ищет похожие слова, 

**4, 10, 11** - наибольший divergence (см. scores)

In [ ]:
print (scores_1)
print (scores_2)
print (scores_3)
print (scores_4)

In [76]:
scores_1 = show_importance_of_transformers_layer(query, answer1)

[CLS] Which is better, a laptop or a desktop? [SEP] Laptop is preferable than desktop because it is more portable. [SEP]
Tokenize the first sentence:
['[CLS]', 'which', 'is', 'better', ',', 'a', 'laptop', 'or', 'a', 'desktop', '?', '[SEP]', 'laptop', 'is', 'prefer', '##able', 'than', 'desktop', 'because', 'it', 'is', 'more', 'portable', '.', '[SEP]']
0 10 25


In [77]:
scores_2 = show_importance_of_transformers_layer(query, answer2)

[CLS] Which is better, a laptop or a desktop? [SEP] If you need portability, the laptop is the best option than desktop. [SEP]
Tokenize the first sentence:
['[CLS]', 'which', 'is', 'better', ',', 'a', 'laptop', 'or', 'a', 'desktop', '?', '[SEP]', 'if', 'you', 'need', 'port', '##ability', ',', 'the', 'laptop', 'is', 'the', 'best', 'option', 'than', 'desktop', '.', '[SEP]']
0 10 28


In [78]:
scores_3 = show_importance_of_transformers_layer(query, answer3)

[CLS] Which is better, a laptop or a desktop? [SEP] The crew was complete: it included a boots, a maker of bonnets and hoods [SEP]
Tokenize the first sentence:
['[CLS]', 'which', 'is', 'better', ',', 'a', 'laptop', 'or', 'a', 'desktop', '?', '[SEP]', 'the', 'crew', 'was', 'complete', ':', 'it', 'included', 'a', 'boots', ',', 'a', 'maker', 'of', 'bonnet', '##s', 'and', 'hood', '##s', '[SEP]']
0 10 31


In [79]:
scores_4 = show_importance_of_transformers_layer(query, answer4)

[CLS] Which is better, a laptop or a desktop? [SEP] Just the pace for snark the Bellman cried [SEP]
Tokenize the first sentence:
['[CLS]', 'which', 'is', 'better', ',', 'a', 'laptop', 'or', 'a', 'desktop', '?', '[SEP]', 'just', 'the', 'pace', 'for', 's', '##nar', '##k', 'the', 'bell', '##man', 'cried', '[SEP]']
0 10 24


See that 3rd head is about finding the same words in sentences

In [ ]:
sentences = ["[CLS] " + query + " [SEP] " + answer1 + " [SEP]"]
print(sentences[0])

# Tokenize with BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_texts[0])
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = (len(tokenizer.tokenize(query)) + 2)*[0] + (len(tokenizer.tokenize(answer1)) + 1)*[1]
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
answ = model(tokens_tensor, segments_tensors)
trans_tokens = tokenized_texts
inp_tokens = tokenized_texts
cls_idx = 0
sep_idx = len(tokenizer.tokenize(query))
end_idx = len(tokenizer.tokenize(query)) + len(tokenizer.tokenize(answer1)) + 3

print (cls_idx, sep_idx, end_idx)
for ii in [3]:
    probs = answ[2][ii][0][0].detach().numpy()

    probs = probs[:len(trans_tokens[0]), :len(inp_tokens[0])]

    fig = pl.figure(x_range=(0, len(inp_tokens[0])), y_range=(0, len(trans_tokens[0])),
                    x_axis_type=None, y_axis_type=None, tools=[])
    fig.image([probs], 0, 0, len(inp_tokens[0]), len(trans_tokens[0]))
    fig.add_layout(bm.LinearAxis(axis_label='source tokens'), 'above')
    fig.xaxis.ticker = np.arange(len(inp_tokens[0])) + 0.5
    fig.xaxis.major_label_overrides = dict(zip(np.arange(len(inp_tokens[0])) + 0.5, inp_tokens[0]))
    fig.xaxis.major_label_orientation = 45

    fig.add_layout(bm.LinearAxis(axis_label='translation tokens'), 'left')
    fig.yaxis.ticker = np.arange(len(trans_tokens[0])) + 0.5
    fig.yaxis.major_label_overrides = dict(zip(np.arange(len(trans_tokens[0])) + 0.5, trans_tokens[0]))
    fig.export_png("3rdlayer.png")
    show(fig)